In [1]:
from google.colab import files
uploaded = files.upload()  # selecciona tu .eim y tu .mp4


Saving worm-pick-assistant-linux-aarch64-v5.eim to worm-pick-assistant-linux-aarch64-v5.eim
Saving motion system.mp4 to motion system.mp4


In [3]:
pip install edge_impulse_linux


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for edge_impulse_linux: filename=edge_impulse_linux-1.2.0-py3-none-any.whl size=11118 sha256=298ce6311e90ad51e50ec4538935206518655b1eab35602db4153cd6d591c55b
  Stored in directory: /root/.cache/pip/wheels/6f/23/47/ac1f05fd2eaa5e8b6714e2248eb01c6c1a4cfd64f465fd5918
Successfully built edge_impulse_linux


In [7]:
!apt-get install portaudio19-dev -y
!pip install pyaudio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 1 to remove and 41 not upgraded.
Need to get 596 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libasound2-dev amd64 1.2.6.1-1ubuntu1 [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack-dev amd64 1:0.125.0-3build2 [206 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/univers

In [14]:
!chmod +x "worm-pick-assistant-linux-x86_64-v6.eim"
!ls -l "worm-pick-assistant-linux-x86_64-v6.eim"

-rwxr-xr-x 1 root root 10649864 Nov 30 03:59 worm-pick-assistant-linux-x86_64-v6.eim


In [26]:
import cv2
import numpy as np
from edge_impulse_linux.image import ImageImpulseRunner

MODEL_PATH = "/content/worm-pick-assistant-linux-x86_64-v6.eim"
INPUT_VIDEO = "/content/motion system.mp4"
OUTPUT_VIDEO = "worms_annotated.mp4"

def annotate_video():
    runner = ImageImpulseRunner(MODEL_PATH)
    try:
        model_info = runner.init()
        print("Model loaded:", model_info['project']['owner'], '/', model_info['project']['name'])

        cap = cv2.VideoCapture(INPUT_VIDEO)
        if not cap.isOpened():
            print("Error: no se pudo abrir el video")
            return

        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (frame_width, frame_height))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # BGR -> RGB
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # El runner hace el resize + preprocesado y devuelve también la imagen redimensionada
            features, resized = runner.get_features_from_image_auto_studio_settings(rgb)
            img_h, img_w, _ = resized.shape  # tamaño real que usa el modelo

            # Inferencia
            result = runner.classify(features)

            # Escalas para pasar del espacio del modelo (img_w x img_h) al frame original
            scale_x = frame_width  / img_w
            scale_y = frame_height / img_h

            # Dibujar bounding boxes reescaladas
            if 'result' in result and 'bounding_boxes' in result['result']:
                for bb in result['result']['bounding_boxes']:
                    # coords en espacio del modelo (img_w x img_h)
                    x = bb['x']
                    y = bb['y']
                    w = bb['width']
                    h = bb['height']

                    # reescalar al tamaño del frame original
                    x1 = int(x * scale_x)
                    y1 = int(y * scale_y)
                    x2 = int((x + w) * scale_x)
                    y2 = int((y + h) * scale_y)

                    label = bb['label']
                    score = bb.get('value', 0.0)

                    if label == 'picktool':
                        color = (0, 0, 255)   # rojo
                    elif label == 'worm':
                        color = (0, 255, 0)   # verde
                    else:
                        color = (255, 255, 255)

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    text = f"{label} {score:.2f}"
                    cv2.rectangle(frame, (x1, y1 - 20), (x2, y1), color, -1)
                    cv2.putText(frame, text, (x1 + 2, y1 - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            out.write(frame)

        cap.release()
        out.release()
        print("Video guardado como:", OUTPUT_VIDEO)

    finally:
        runner.stop()

annotate_video()


Model loaded: jefferson.sarmiento / worm pick assistant
Video guardado como: worms_annotated.mp4
